# Time series sliding

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("daily-total-female-births.csv")

In [3]:
df.head()

,Date,Births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [19]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Sequence of observations as a list or NumPy array.
    n_in: Number of lag observations as input (X).
    n_out: Number of observations as output (y).
    dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    if type(data) is list:
        df = DataFrame(data) 
    else:
        df = data     
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [22]:
data = series_to_supervised(df)

In [23]:
data

,var1(t-1),var2(t-1),var1(t),var2(t)
1,1959-01-01,35.0,1959-01-02,32
2,1959-01-02,32.0,1959-01-03,30
3,1959-01-03,30.0,1959-01-04,31
4,1959-01-04,31.0,1959-01-05,44
5,1959-01-05,44.0,1959-01-06,29
...,...,...,...,...
360,1959-12-26,34.0,1959-12-27,37
361,1959-12-27,37.0,1959-12-28,52
362,1959-12-28,52.0,1959-12-29,48
363,1959-12-29,48.0,1959-12-30,55
